In [1]:
import os
SPARK_HOME="/opt/spark"
os.environ["PYSPARK_PYTHON"] = "python3"
# os.environ["PYSPARK_PYTHON"]="python"
os.environ["PYSPARK_DRIVER_PYTHON"]="jupyter"
os.environ["PYSPARK_DRIVER_PYTHON_OPTS"]="notebook"
os.environ["SPARK_HOME"] = SPARK_HOME
os.environ["SPARK_CLASSPATH"] = "/opt/spark/jars/sqljdbc4.jar"

 

from pyspark.sql import SparkSession
spark = SparkSession \
        .builder \
        .appName("Task_3") \
        .getOrCreate()

In [2]:
#from pyspark.sql import SQLContext
# sqlContext = SQLContext(spark)
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [3]:
df_path = "/home/rupeshr/Desktop/TSA_Python/dataset/multivariate_dataset.csv"
df =spark.read.format("com.databricks.spark.csv")\
                    .option("multiline", "true")\
                    .option("header", "true")\
                    .option("inferSchema", "true")\
                    .option("sep", ',')\
                    .option('escape', '\"')\
                    .option("allowSingleQuotes", "true")\
                    .option("ignoreLeadingWhiteSpace", "true")\
                    .option("ignoreTrailingWhiteSpace", "true")\
                    .load(df_path)
df.printSchema()
df.show()

root
 |-- Date: string (nullable = true)
 |-- Money: double (nullable = true)
 |-- Spending: double (nullable = true)
 |-- Gain: integer (nullable = true)
 |-- Loss: integer (nullable = true)

+----------+------+--------+----+----+
|      Date| Money|Spending|Gain|Loss|
+----------+------+--------+----+----+
|1995-01-01|3492.4|  4851.2|2891|1000|
|1995-02-01|3489.9|  4850.8|2893|1101|
|1995-03-01|3491.1|  4885.4|2895|1202|
|1995-04-01|3499.2|  4890.2|2897|1303|
|1995-05-01|3524.2|  4933.1|   0|1404|
|1995-06-01|3548.9|  4977.5|   0|1505|
|1995-07-01|3567.4|  4970.2|   0|1606|
|1995-08-01|3589.0|  5005.3|   0|1707|
|1995-09-01|3602.1|  5020.5|   0|1808|
|1995-10-01|3613.4|  5013.9|   0|1909|
|1995-11-01|3619.9|  5055.6|   0|2010|
|1995-12-01|3629.5|  5097.5|   0|2111|
|1996-01-01|3647.9|  5085.7|   0|2212|
|1996-02-01|3661.9|  5132.8|2917|2313|
|1996-03-01|3687.0|  5173.3|2919|2414|
|1996-04-01|3697.8|  5208.0|2921|   0|
|1996-05-01|3709.7|  5223.8|2923|   0|
|1996-06-01|3722.7|  5229.8

In [4]:
import pandas as pd
from pyspark.sql.types import *
from pyspark.sql import Window
# Auxiliary functions
# Pandas Types -> Spark Types
#from edge.stats.sparksession import spark_session

def equivalent_type(f):
    if f == 'datetime64[ns]': return DateType()
    elif f == 'int64': return LongType()
    elif f == 'int32': return IntegerType()
    elif f == 'float64': return DoubleType()
    else: return StringType()
def define_structure(string, format_type):
    try: typo = equivalent_type(format_type)
    except: typo = StringType()
    return StructField(string, typo)
# Converts pandas dataframe to spark dataframe
def pandas_to_spark(df_pandas):
    sqlContext = spark
    columns = list(df_pandas.columns)
    types = list(df_pandas.dtypes)
    struct_list = []
    for column, typo in zip(columns, types):
        struct_list.append(define_structure(column, typo))
    p_schema = StructType(struct_list)
    return sqlContext.createDataFrame(df_pandas, p_schema)


def replace(df, column,replace,value=None): 
    for i in df.columns:
        if i in column:
            s = df.select(i).dtypes[0]
            if s[1] == "int" or s[1] == "float" or s[1] == "double" or s[1] == "bigint" or s[1] == "long":
                if replace =='mean':
                    mean = df.select(avg(i)).collect()[0]
                    mean = mean.asDict()
                    s="avg(%s)"%i
                    mean = mean[s]
                    df= df.na.replace(to_replace=0,value=mean,subset=i)
                elif replace =='median':
                    median = df.approxQuantile(i, [0.5], 0)
                    median = median[0]
                    df= df.na.replace(to_replace=0,value=median,subset=i)
                elif replace =='max':
                    maximum = df.select(max(i)).collect()[0]
                    maximum = maximum.asDict()
                    s="max(%s)"%i
                    maximum = maximum[s]
                    df= df.na.replace(to_replace=0,value=maximum,subset=i)
                elif replace =='min':
                    rem_zero = df.filter((df[i] != 0))
                    rem_zero=rem_zero.drop('index')
                    minimum = rem_zero.select(min(i)).collect()[0]
                    minimum = minimum.asDict()
                    s="min(%s)"%i
                    minimum = minimum[s]
                    df= df.na.replace(to_replace=0,value=minimum,subset=i)
                    df=df.drop('index')
                elif replace =='ffill':
                    temp_df=df.select(i).toPandas()
                    temp_df.replace(0,method='ffill',inplace=True)
                    temp_df=pandas_to_spark(temp_df) 
                    temp_df=temp_df.select(col(i).alias('renamed'))
                    w = Window.orderBy(monotonically_increasing_id())
                    df = df.withColumn("index", row_number().over(w))
                    temp_df = temp_df.withColumn("index", row_number().over(w))
                    df= df.join(temp_df, df['index'] == temp_df['index'], 'left').drop(temp_df['index'])
                    df=df.withColumn(i,col('renamed')).drop('index','renamed')
                elif replace =='bfill':
                    temp_df=df.select(i).toPandas()
                    temp_df.replace(0,method='bfill',inplace=True)
                    temp_df=pandas_to_spark(temp_df) 
                    temp_df=temp_df.select(col(i).alias('renamed'))
                    w = Window.orderBy(monotonically_increasing_id())
                    df = df.withColumn("index", row_number().over(w))
                    temp_df = temp_df.withColumn("index", row_number().over(w))
                    df= df.join(temp_df, df['index'] == temp_df['index'], 'left').drop(temp_df['index'])
                    df=df.withColumn(i,col('renamed')).drop('index','renamed')
                elif replace =='custom':
                    df= df.na.replace(to_replace=0,value=value,subset=i)
                    
    return df

df1=replace(df,'Gain','bfill')
df1.show()

+----------+------+--------+----+----+
|      Date| Money|Spending|Gain|Loss|
+----------+------+--------+----+----+
|1995-01-01|3492.4|  4851.2|2891|1000|
|1995-02-01|3489.9|  4850.8|2893|1101|
|1995-03-01|3491.1|  4885.4|2895|1202|
|1995-04-01|3499.2|  4890.2|2897|1303|
|1995-05-01|3524.2|  4933.1|2917|1404|
|1995-06-01|3548.9|  4977.5|2917|1505|
|1995-07-01|3567.4|  4970.2|2917|1606|
|1995-08-01|3589.0|  5005.3|2917|1707|
|1995-09-01|3602.1|  5020.5|2917|1808|
|1995-10-01|3613.4|  5013.9|2917|1909|
|1995-11-01|3619.9|  5055.6|2917|2010|
|1995-12-01|3629.5|  5097.5|2917|2111|
|1996-01-01|3647.9|  5085.7|2917|2212|
|1996-02-01|3661.9|  5132.8|2917|2313|
|1996-03-01|3687.0|  5173.3|2919|2414|
|1996-04-01|3697.8|  5208.0|2921|   0|
|1996-05-01|3709.7|  5223.8|2923|   0|
|1996-06-01|3722.7|  5229.8|2925|   0|
|1996-07-01|3737.3|  5251.9|2927|   0|
|1996-08-01|3744.3|  5275.0|2929|   0|
+----------+------+--------+----+----+
only showing top 20 rows



In [ ]:

df = pd.DataFrame({"Col1": [10, 20, 15, 30, 45],
                   "Col2": [13, 23, 18, 33, 48],
                   "Col3": [17, 27, 22, 37, 52]},
                  index=pd.date_range("2020-01-01", "2020-01-05"))